In [26]:
import pandas as pd
import json
from datetime import datetime
import re

# JSON 데이터 로드 (문자열로 저장된 JSON 데이터라면 json.loads()를 사용)
with open("IPOSTOCK_data.json", "r", encoding="utf-8-sig") as file:
    json_data = json.load(file)  # JSON 파일을 로드


# 연도별 column
yearly_columns=[]


for i in list(json_data[0].values())[0].get("재무정보",{}):
  yearly_columns.append(i)
yearly_columns.append("영업수익")

# 데이터를 담을 리스트 초기화
data_list = []

# JSON을 반복하며 필요한 정보 추출
for company_data in json_data:
    부채_list=[]
    유동_list=[]
    영업이익_list=[]
    당기순이익_list=[]
    매출액_list=[]
    ROE_list=[]
    EPS_list=[]
    EV_list=[]
    for company_name, details in company_data.items():
      date_str = details.get("공모정보", {}).get("상장일", None)
      full_date=datetime.strptime(date_str,"%Y.%m.%d").strftime("%Y-%m-%d")
      year_key = full_date[:4]
      year,month=full_date[:7].split("-")

      a=re.findall(r'\d*\.?\d+',details.get("수요예측", {}).get("의무보유확약비율", None))
      aa=a[0] if a else 0

      수요예측일_raw = details.get("공모정보", {}).get("수요예측일", None)

      if 수요예측일_raw:
          # "2025.01.09 ~ 01.15" 형식의 문자열 처리
          dates = 수요예측일_raw.split(" ~ ")
          start_date = dates[0]  # "2025.01.09"
          end_date = dates[1]    # "01.15"

          # 종료일에 연도 추가 (시작일의 연도 사용)
          year = start_date.split(".")[0]
          end_date = f"{year}.{end_date}"

          # 날짜 포맷 변경 (YYYY.MM.DD -> YYYY-MM-DD)
          수요예측_시작일 = start_date.replace(".", "-")
          수요예측_종료일 = end_date.replace(".", "-")
      else:
          수요예측_시작일 = None
          수요예측_종료일 = None

      if details.get("재무정보")=={}:
        continue
      for i in range(2):
        row = {
            "기업명": company_name,
            "year_key": str(int(year_key)-i),
            "month_key": f"{int(year)-i}-{month}",
            "(희망)공모가격 하한가": [int(p.replace(",","")) for p in re.findall(r"\d{1,3}(?:,\d{3})*",details.get("수요예측", {}).get("(희망)공모가격", None))][0],
            "(희망)공모가격 상한가": [int(p.replace(",","")) for p in re.findall(r"\d{1,3}(?:,\d{3})*",details.get("수요예측", {}).get("(희망)공모가격", None))][1],
            "단순 기관경쟁률": re.findall(r"\d+\.\d+|\d+",details.get("수요예측", {}).get("단순 기관경쟁률", None))[0],
            "의무보유확약비율": aa,
            "(확정)공모가격": re.findall(r"\d{1,3}(?:,\d{3})*",details.get("공모정보", {}).get("(확정)공모가격", None))[0],
            "청약경쟁률": int(re.search(r'^\d+',details.get("공모정보", {}).get("청약경쟁률", None)).group()),
            "수요예측 시작일": 수요예측_시작일,
            "수요예측 종료일": 수요예측_종료일,
            "상장일": details.get("공모정보", {}).get("상장일", None).replace(".", "-"),
            "공모후 발행주식수": int(re.search(r'\d{1,3}(?:,\d{3})*',details.get("주주구성", {}).get("공모후 발행주식수", None)).group().replace(",", "")),
            "유동자산": details.get("재무정보",{}).get("유동자산",None)[i],
            "비유동자산": details.get("재무정보",{}).get("비유동자산",None)[i],
            "유동부채": details.get("재무정보",{}).get("유동부채",None)[i],
            "비유동부채": details.get("재무정보",{}).get("비유동부채",None)[i],
            "자본금": details.get("재무정보",{}).get("자본금",None)[i],
            "이익잉여금": details.get("재무정보",{}).get("이익잉여금",None)[i],
            "기타자본항목": details.get("재무정보",{}).get("기타자본항목",None)[i],
            "영업이익": details.get("재무정보",{}).get("영업이익",None)[i],
            "당기순이익": details.get("재무정보",{}).get("당기순이익",None)[i],
            "종가대비등락율": details.get("종가대비등락율", None),
        }
        try:
          row["매출액"]=details.get("재무정보",{}).get("매출액")[i]
        except:
          row["매출액"]=details.get("재무정보",{}).get("영업수익")[i]
        매출액_list.append(row["매출액"])

        # ✅ 재무 비율 계산 (0으로 나누는 오류 방지)
        자본총계 = row["자본금"] + row["이익잉여금"] + row["기타자본항목"]
        부채총계 = row["유동부채"] + row["비유동부채"]

        row["부채비율"] = (부채총계 / 자본총계) if 자본총계 else 0
        부채_list.append(row["부채비율"])
        row["유동비율"] = (row["유동자산"] / row["유동부채"]) if row["유동부채"] else 0
        유동_list.append(row["유동비율"])
        row["영업이익률"] = (row["영업이익"] / row["매출액"]) if row["매출액"] else 0
        영업이익_list.append(row["영업이익률"])
        row["당기순이익률"] = (row["당기순이익"] / row["매출액"]) if row["매출액"] else 0
        당기순이익_list.append(row["당기순이익률"])
        row["ROE"] = (row["당기순이익"] / 자본총계) if 자본총계 else 0
        ROE_list.append(row["ROE"])
        row["EPS"] = (row["당기순이익"] / row["공모후 발행주식수"]) if row["공모후 발행주식수"] else 0
        EPS_list.append(row["EPS"])
        row["EV/영업이익"] = ((row["유동자산"] + row["비유동자산"] - 부채총계) / row["영업이익"]) if row["영업이익"] else 0
        EV_list.append(row["EV/영업이익"])

        # 최대주주 소유주 비율
        target=details.get("주주구성",{}).get("주주구성 table",{}).get("보호예수매도금지",{})
        row["최대주주 소유주 비율"]=float(re.sub(r"[^\d.]", "", target[list(target.keys())[0]][2]))

        # 최대주주 보호예수 기간
        target=details.get("주주구성",{}).get("주주구성 table",{}).get("보호예수매도금지",{})
        row["최대주주 보호예수 기간"]=int(re.findall(r"\d+",target[list(target.keys())[0]][3])[0])

        # 보호예수 비율
        target=details.get("주주구성",{}).get("주주구성 table",{}).get("보호예수매도금지",{})
        row["보호예수 비율"]=float(re.sub(r"[^\d.]", "",target[list(target.keys())[-1]][2]))

        # 부채비율 변화율
        if i==1:
          row["부채비율 변화율"]=부채_list[0]-부채_list[1]
          row["유동비율 변화율"]=유동_list[0]-유동_list[1]
          row["영업이익률 변화율"]=영업이익_list[0]-영업이익_list[1]
          row["당기순이익률 변화율"]=당기순이익_list[0]-당기순이익_list[1]
          row["매출액 변화율"]=매출액_list[0]-매출액_list[1]
          row["ROE 변화율"]=ROE_list[0]-ROE_list[1]
          row["EPS 변화율"]=EPS_list[0]-EPS_list[1]
          row["EV/영업이익 변화율"]=EV_list[0]-EV_list[1]
        data_list.append(row)

# 리스트를 DataFrame으로 변환
df = pd.DataFrame(data_list)

df["부채비율 변화율"] = df["부채비율 변화율"].fillna(df["부채비율 변화율"].shift(-1))
df["유동비율 변화율"] = df["유동비율 변화율"].fillna(df["유동비율 변화율"].shift(-1))
df["영업이익률 변화율"] = df["영업이익률 변화율"].fillna(df["영업이익률 변화율"].shift(-1))
df["당기순이익률 변화율"] = df["당기순이익률 변화율"].fillna(df["당기순이익률 변화율"].shift(-1))
df["매출액 변화율"] = df["매출액 변화율"].fillna(df["매출액 변화율"].shift(-1))
df["ROE 변화율"] = df["ROE 변화율"].fillna(df["ROE 변화율"].shift(-1))
df["EPS 변화율"] = df["EPS 변화율"].fillna(df["EPS 변화율"].shift(-1))
df["EV/영업이익 변화율"] = df["EV/영업이익 변화율"].fillna(df["EV/영업이익 변화율"].shift(-1))

df=df.sort_values(by=['기업명','year_key'], ascending=[True,False]).drop_duplicates(subset=['기업명'],keep='first')

In [27]:

# DataFrame 출력
print(df)
print(yearly_columns)
# CSV로 저장하고 싶다면 아래 코드 실행
df.to_csv("X_stat.csv", index=False, encoding="utf-8-sig")

                기업명 year_key month_key  (희망)공모가격 하한가  (희망)공모가격 상한가 단순 기관경쟁률  \
1192      DSC인베스트먼트     2016   2016-12          2500          2800     11.2   
1238  GRT(그레이트리치과기)     2016   2016-10          4000          6000   140.75   
1162        ING생명보험     2017   2017-05         31500         40000     3.97   
1044            JTC     2018   2018-04          6200          7600      478   
1236         JW생명과학     2016   2016-10         27000         32500      225   
...             ...      ...       ...           ...           ...      ...   
1018           휴네시온     2018   2018-08          8700         10000   751.89   
1474           휴메딕스     2014   2014-12         22700         25500    688.8   
1336             휴젤     2015   2015-12        190000        210000      6.7   
1416         흥국에프엔비     2015   2015-08         14500         17500   666.88   
1146             힘스     2017   2017-07         15500         18800   729.64   

     의무보유확약비율 (확정)공모가격  청약경쟁률    수요예측 시작일  ... 최대주주

In [28]:
import pandas as pd
import json
import os

# Colab에 업로드한 파일들을 사용하기 위해 경로 지정
file_names = [
    "KOREA_국고채_금리_(3년).json",
    "KOREA_국고채_금리_(10년).json",
    "KOREA_기업경기실사지수(BSI).json",
    "KOREA_기준금리.json",
    "KOREA_무역수지.json",
    "KOREA_산업생산지수(IPI).json",
    "KOREA_생산자물가_상승률_(PPI).json",
    "KOREA_소비심리지수(CSI).json",
    "KOREA_소비자물가_상승률_(CPI).json",
    "KOREA_수출입_물량지수.json",
    "KOREA_외환보유액.json",
    "KOREA_원달러_환율.json",
    "KOREA_통화량_(M2).json"
]

merged_df = pd.DataFrame()

for file_name in file_names:
    file_path = os.path.join(file_name)  # Colab 기본 경로는 /content/

    with open(file_path, "r", encoding="utf-8-sig") as f:
        data = json.load(f)

    df = pd.DataFrame(data)

    # 날짜를 기준으로 병합 (outer join)
    if merged_df.empty:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on="날짜", how="outer")

# 날짜 기준으로 정렬
merged_df = merged_df.sort_values(by="날짜").reset_index(drop=True)
merged_df["날짜"] = pd.to_datetime(merged_df["날짜"], format="%Y%m").dt.strftime("%Y-%m")
merged_df = merged_df.rename(columns={"날짜": "month_key"})


In [29]:
print(merged_df.head())

  month_key 국고채 금리 (3년) 국고채 금리 (10년) 기업경기실사지수(BSI) 기준금리    무역수지 산업생산지수(IPI)  \
0   2014-01       2.892        3.647            74  2.5  5993.1          89   
1   2014-01       2.892        3.647            74  2.5  5993.1        92.1   
2   2014-02       2.854        3.522            76  2.5  6803.9          85   
3   2014-02       2.854        3.522            76  2.5  6803.9        91.5   
4   2014-03       2.874        3.543            80  2.5  5420.6        93.7   

  생산자물가 상승률 (PPI) 소비심리지수(CSI) 소비자물가 상승률 (CPI) 수출입 물량지수      외환보유액   원달러 환율  \
0          101.57       110.2           93.73   5993.1  348393811  1064.75   
1          101.57       110.2           93.73   5993.1  348393811  1064.75   
2           101.7       109.2           93.98   6803.9  351791760   1071.3   
3           101.7       109.2           93.98   6803.9  351791760   1071.3   
4          101.66       109.7          94.153   5420.6  354341349  1070.89   

    통화량 (M2)  
0  1937700.8  
1  1937700.8  
2  1957315.

In [30]:
merged_df.to_csv("df_monthly.csv", index=False, encoding="utf-8-sig")


In [31]:
import pandas as pd
import json
import os

# Colab에 업로드한 파일들을 사용하기 위해 경로 지정
file_names = [
    "KOSPI_1001.json",
    "KOSPI_1005.json",
    "KOSPI_1008.json",
    "KOSPI_1011.json",
    "KOSPI_1012.json",
    "KOSPI_1013.json",
    "KOSPI_1014.json",
    "KOSPI_1015.json",
    "KOSPI_1017.json",
    "KOSPI_1018.json",
    "KOSPI_1020.json",
    "KOSPI_1021.json",
    "KOSPI_1024.json",
    "KOSPI_1026.json",
    "KOSPI_1035.json",
    "KOSPI_1155.json",
    "KOSPI_1156.json"
]

# 빈 데이터프레임 생성
merged_df_daily = pd.DataFrame()

for file_name in file_names:
    file_path = os.path.join(file_name)  # Colab 기본 경로는 /content/

    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    df = pd.DataFrame(data)[["날짜", "종가"]]  # "날짜"와 "종가"만 선택

    # 종목명을 컬럼명으로 변경 (ex: KOSPI_1001.json → KOSPI_1001)
    stock_name = file_name.replace(".json", "")
    df = df.rename(columns={"종가": stock_name})

    # 날짜를 기준으로 병합 (outer join)
    if merged_df_daily.empty:
        merged_df_daily = df
    else:
        merged_df_daily = pd.merge(merged_df_daily, df, on="날짜", how="outer")

# 날짜 기준으로 정렬
merged_df_daily = merged_df_daily.sort_values(by="날짜").reset_index(drop=True)

# 날짜 형식 변환 (YYYY-MM-DD 형식)
merged_df_daily = merged_df_daily.rename(columns={"날짜": "date"})
# "year_key" 컬럼 추가 (연도만 추출)
merged_df_daily["year_key"] = pd.to_datetime(merged_df_daily["date"]).dt.year

# "month_key" 컬럼 추가 (연-월 형식으로 변환)
merged_df_daily["month_key"] = pd.to_datetime(merged_df_daily["date"]).dt.strftime("%Y-%m")


In [32]:
print(merged_df_daily.head())

         date  KOSPI_1001  KOSPI_1005  KOSPI_1008  KOSPI_1011  KOSPI_1012  \
0  2014-01-02     1967.19     3837.31     4071.75     5093.80      932.94   
1  2014-01-03     1946.14     3774.62     4034.17     5078.84      927.42   
2  2014-01-06     1953.28     3791.71     4037.53     5032.36      932.93   
3  2014-01-07     1959.44     3779.62     4042.91     5015.49      935.40   
4  2014-01-08     1958.96     3789.14     4029.45     4995.20      928.99   

   KOSPI_1013  KOSPI_1014  KOSPI_1015  KOSPI_1017  KOSPI_1018  KOSPI_1020  \
0    10259.32     1808.19     2553.37     1174.31      127.99      331.56   
1    10194.44     1823.65     2532.40     1167.01      125.75      328.52   
2    10279.95     1839.27     2552.89     1194.94      122.14      331.33   
3    10289.69     1837.06     2584.20     1204.47      122.50      330.70   
4    10242.71     1817.50     2584.89     1202.12      124.19      329.86   

   KOSPI_1021  KOSPI_1024  KOSPI_1026  KOSPI_1035  KOSPI_1155  KOSPI_1156 

In [33]:
merged_df_daily.to_csv("df_daily.csv", index=False, encoding="utf-8-sig")


In [36]:
df_daily = pd.read_csv("df_daily.csv",encoding="utf-8-sig")
df_monthly = pd.read_csv("df_monthly.csv",encoding="utf-8-sig")

# month_key를 기준으로 df_daily에 df_monthly를 병합
df_merged = df_daily.merge(df_monthly, on="month_key", how="left")


df_merged.head()

,date,KOSPI_1001,KOSPI_1005,KOSPI_1008,KOSPI_1011,KOSPI_1012,KOSPI_1013,KOSPI_1014,KOSPI_1015,KOSPI_1017,...,기준금리,무역수지,산업생산지수(IPI),생산자물가 상승률 (PPI),소비심리지수(CSI),소비자물가 상승률 (CPI),수출입 물량지수,외환보유액,원달러 환율,통화량 (M2)
0,2014-01-02,1967.19,3837.31,4071.75,5093.80,932.94,10259.32,1808.19,2553.37,1174.31,...,2.5,5993.1,89.0,101.57,110.2,93.73,5993.1,348393811.0,1064.75,1937700.8
1,2014-01-02,1967.19,3837.31,4071.75,5093.80,932.94,10259.32,1808.19,2553.37,1174.31,...,2.5,5993.1,92.1,101.57,110.2,93.73,5993.1,348393811.0,1064.75,1937700.8
2,2014-01-03,1946.14,3774.62,4034.17,5078.84,927.42,10194.44,1823.65,2532.40,1167.01,...,2.5,5993.1,89.0,101.57,110.2,93.73,5993.1,348393811.0,1064.75,1937700.8
3,2014-01-03,1946.14,3774.62,4034.17,5078.84,927.42,10194.44,1823.65,2532.40,1167.01,...,2.5,5993.1,92.1,101.57,110.2,93.73,5993.1,348393811.0,1064.75,1937700.8
4,2014-01-06,1953.28,3791.71,4037.53,5032.36,932.93,10279.95,1839.27,2552.89,1194.94,...,2.5,5993.1,89.0,101.57,110.2,93.73,5993.1,348393811.0,1064.75,1937700.8


In [37]:
df_merged.to_csv("df_merged.csv", index=False, encoding='utf-8-sig')

In [38]:
df_merged = pd.read_csv("df_merged.csv", encoding="utf-8-sig")
df_yearly = pd.read_csv("X_stat.csv", encoding="utf-8-sig")

# year_key를 기준으로 df_merged에 df_yearly를 병합
df_final = df_merged.merge(df_yearly, on="year_key", how="left")

# 결과 출력
print(df_final.head())



         date  KOSPI_1001  KOSPI_1005  KOSPI_1008  KOSPI_1011  KOSPI_1012  \
0  2014-01-02     1967.19     3837.31     4071.75      5093.8      932.94   
1  2014-01-02     1967.19     3837.31     4071.75      5093.8      932.94   
2  2014-01-02     1967.19     3837.31     4071.75      5093.8      932.94   
3  2014-01-02     1967.19     3837.31     4071.75      5093.8      932.94   
4  2014-01-02     1967.19     3837.31     4071.75      5093.8      932.94   

   KOSPI_1013  KOSPI_1014  KOSPI_1015  KOSPI_1017  ...  최대주주 보호예수 기간  보호예수 비율  \
0    10259.32     1808.19     2553.37     1174.31  ...             6    58.59   
1    10259.32     1808.19     2553.37     1174.31  ...             6    74.69   
2    10259.32     1808.19     2553.37     1174.31  ...             6    56.30   
3    10259.32     1808.19     2553.37     1174.31  ...             6    82.76   
4    10259.32     1808.19     2553.37     1174.31  ...             1    73.25   

   부채비율 변화율  유동비율 변화율  영업이익률 변화율  당기순이익률 변화율      

In [39]:
# 병합된 데이터 저장
df_final.to_csv("df_final.csv", encoding="utf-8-sig", index=False)